In [1]:
import itertools
import math
import random
import numpy

# Step 1
Generate Rankings of Relevance for E and P


In [2]:
#Generating all possible combinations of relevance,
#where  0 - N, 1 - R, 2 - HR
possible_rankings = itertools.product([0,1,2], repeat=5)
P = [list(i) for i in possible_rankings]
E = P[:]

#Constructing all possible pairs
L = []
for item1 in P:
    for item2 in E:
        pair = []
        pair.append(item1)
        pair.append(item2)
        L.append(pair)
print "Number of possible rankings: ", len(L)

Number of possible rankings:  59049


# Step 2
Implement Evaluation Measures 

In [3]:
def precision(res, k):
    counter = 0
    for i in range(k):
        if res[i] != 0:
            counter += 1
    return 1.0*counter / k

def DCG(res, k):
    sum = 0
    for r in range(1,k+1):
        sum += (2**res[r-1] - 1)/math.log(1+r,2)
    return sum

def RBP(res, theta = 0.8):
    sum = 0
    for k in range(1, len(res)):
        sum += res[k-1] * theta**(k-1)*(1 - theta)
    return sum

N=1165
print "Example:"
print " Rankings: ", L[N]
print " Precision@4: ", [precision(L[N][0],k = 4), precision(L[N][1],k = 4)]
print " DCG@4: ", [DCG(L[N][0],k = 3),DCG(L[N][1],k = 4)]
print " RBP: ", [RBP(L[N][0]), RBP(L[N][1])]

Example:
 Rankings:  [[0, 0, 0, 1, 1], [2, 1, 0, 1, 1]]
 Precision@4:  [0.25, 0.75]
 DCG@4:  [0.0, 4.061606311644851]
 RBP:  [0.1024, 0.6623999999999999]


# Step 3
 Calculate the 𝛥measure

In [8]:
def delta_measure(pairs, measure, k=5):
    result = []
    for pair in pairs:
        if measure == "precision":
            P = precision(pair[0], k)
            E = precision(pair[1], k)
        
        if measure == "DCG":
            P = DCG(pair[0], k)
            E = DCG(pair[1], k)
            
        if measure == "RBP":
            P = RBP(pair[0])
            E = RBP(pair[1])
         
        diff = E - P
        
        if diff > 0:
            result.append([pair, diff])
            
    return result

dcg_res = delta_measure(L, "DCG")
prec_res = delta_measure(L, "precision")
rbp_res = delta_measure(L, "RBP")

N=5432
print "Example: "
print " Rankings: ", res[N][0]
print " DCG difference: ", res[N][1]

print "Number of pairs for which E outperforms P for different measures:"
print " Precision:", len(prec_res)
print " DCG:", len(dcg_res)
print " RBP:", len(rbp_res)

Example: 
 Rankings:  [[0, 0, 2, 2, 1], [2, 0, 2, 2, 2]]
 DCG difference:  3.77370561447
Number of pairs for which E outperforms P for different measures:
 Precision: 21762
 DCG: 29376
 RBP: 29160


# Step 4
Implement Interleaving

In our implementations of the generic Team Draft Interleaving and Probabilistic Interleaving, we also consider overlapping documents between the two rankings. To find which documents are the same, we are comparing the labels of the documents.

To make use of this generic algorithm with our rankings, that have relevance as labels, we have to alter the labels so that each one of them is unique. We do that by appending a unique number next to the relevance. For example, the rankings pair:

[0, 0, 2, 2, 1], [2, 0, 2, 2, 2]

becomes

[00, 01, 22, 23, 14], [25, 06, 27, 28, 29]

In this way, we make sure that there are no overlapping documents between the two rankings, as is required by Note 5 a).

In [65]:
#Team Draft Interleaving

#make all labels of the selected pair of rankings unique
def preprocess(pair):
    c = 0
    new_pair = ([], [])
    for i in range(2):
        for item in pair[i]:
            new_pair[i].append(str(item)+ str(c))
            c+=1
    return new_pair


def outcome_function(c,a):
    counters = [0, 0]
    for i in c:
        counters[a[i]]+=1
    return numpy.sign(counters[0]-counters[1])


#generate a list of random clicks
def get_clicks(l, number):
    ind_order = range(len(l))
    random.shuffle(ind_order)
    return ind_order[:number]

#RCM stands for Random Click Model, in which a document has a probability 'r' of being clicked (r = parameters[0])
#SDCM stands for Simple Dependent Click Model (lambdas = parameters)
#default means that a certain amount of random docs will be clicked at random(#clicked docs = parameters[0], default is 2)
def simulate_clicks(interleaved,  click_model = "default", parameters = [2], clicks = None):
    if clicks == None:
        if click_model == "default":
            clicks = get_clicks(interleaved, parameters[0])
        if click_model == "RCM":
            clicks = RCM(interleaved,parameters[0])
        if click_model == "SDCM":
            attr  = get_attractiveness(interleaved)
            clicks = SDCM(attr ,parameters)
    
    return clicks

#a) create the interleaving list
def team_draft_interleave(pair):
    lists = [pair[0][:],pair[1][:]]
    l = []
    a = []
    while lists[0] or lists[1]:
        if lists[0] and lists[1]:
            first = random.randint(0,1)
        elif lists[0]:
            first = 0
        else:
            first = 1
        second = int(math.fabs(first-1))

        #pick from the first list
        doc = lists[first][0]
        l.append(doc)
        a.append(first)
        #delete the doc from the first list (top element) and the second list
        del lists[first][0]   
        try:
            lists[second].remove(doc)     
        except:
            pass
            
        #pick from the second list
        if lists[second]:
            doc = lists[second][0]
            l.append(doc)
            a.append(second) 
            #delete the doc from the second list (top element) and the first list
            del lists[second][0]        
            try:
                lists[first].remove(doc)    
            except:
                pass

    res = []
    for i in range(len(l)):
        res.append([l[i],a[i]])
    return res #l,a
    
#b) evaluate the interleaved list
def team_draft_evaluate(interleaved, clicks):
    l, a = zip(*interleaved)
    o = outcome_function(clicks,a)
    return o

def team_draft(pair, click_model = "default", parameters = [2], clicks = None):
    interleaved = team_draft_interleave(pair)
    clicks = simulate_clicks(interleaved,  click_model = click_model, parameters = parameters, clicks = clicks)
    l, a = zip(*interleaved)
    evaluation = team_draft_evaluate(interleaved, clicks)
    print " Interleaved:", [int(i[0]) for i in l]
    print " Assignments:", list(a)
    print " Clicks:", clicks
    print " Comparison:", evaluation
    return evaluation

N=1232
print "--TeamDraft Interleaving--"
print "Rankings: ", L[N]
#res = team_draft(preprocess(L[N]))
print "-RCM:"
res = team_draft(preprocess(L[N]),click_model = "RCM", parameters = [r])
print "-SDCM:"
res = team_draft(preprocess(L[N]),click_model = "SDCM", parameters = lambdas)
#print " Evaluation: ", res

--TeamDraft Interleaving--
Rankings:  [[0, 0, 0, 1, 2], [0, 0, 1, 2, 2]]
-RCM:
 Interleaved: [0, 0, 0, 0, 0, 1, 1, 2, 2, 2]
 Assignments: [1, 0, 0, 1, 0, 1, 0, 1, 0, 1]
 Clicks: []
 Comparison: 0
-SDCM:
 Interleaved: [0, 0, 0, 0, 1, 0, 2, 1, 2, 2]
 Assignments: [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
 Clicks: [4]
 Comparison: -1


In [40]:
#Probabilistic Interleaving

#a) create the interleaving list
def prob_interleave(pair, tau = 3):
    #add to each entry of a ranking the id of the ranking it is from
    lists = [zip(pair[0],[0]*len(pair[0])), zip(pair[1],[1]*len(pair[1]))]
    result = []
    sum = 0
    for i in range(0, len(lists[0])):
        sum += 1.0/((i+1)**tau)
    p = [[(1.0/((i+1)**tau))/sum for i in range(0, len(lists[0]))], [(1.0/((i+1)**tau))/sum for i in range(0, len(lists[0]))]]

    counter = [len(lists[0]),len(lists[1])] # the number of documents left in l1 and l2 
    
    #construct the interleaving by iteratively
    #adding a pair of entries to the resulting list
    while counter[0] > 0 or counter[1] > 0:
        #select the id of the ranking to draw an entry from
        if counter[0] > 0 and counter[1] > 0:
            selected = random.randint(0,1)
        elif counter[0] > 0:
            selected = 0
        else:
            selected = 1
        other_ind = int(math.fabs(selected-1))
        
        #select a new entry with the softmax
        doc_index = numpy.random.choice(range(len(lists[selected])),1,p=p[selected])[0]
        doc = lists[selected][doc_index]
        result.append(doc)
        
        #renormalize the softmax probabilities (do not renormalize if there are no documents left)
        oldp = p[selected][doc_index]
        p[selected][doc_index] = 0
        counter[selected] -= 1
        if counter[selected] > 0: 
            p[selected] = [val/(1.0-oldp) for val in p[selected]]
        
        doc_l = list(doc)
        doc_l[1] = other_ind
        doc = (doc_l[0], doc_l[1])
        if doc in lists[other_ind]:
            ind = lists[other_ind].index(doc)
            oldp = p[other_ind][ind]
            p[other_ind][ind] = 0
            counter[other_ind] -= 1
            if counter[other_ind] > 0:
                p[other_ind] = [val/(1.0-oldp) for val in p[other_ind]]
        
        
    return result
        
    
#b) evaluate the interleaved list
def prob_evaluate(interleaved, clicks):
    return team_draft_evaluate(interleaved, clicks)
   

def prob(pair,  click_model = "default", parameters = [2], clicks = None):
    interleaved = prob_interleave(pair) 
    print interleaved
    clicks = simulate_clicks(interleaved,  click_model = click_model, parameters = parameters, clicks = clicks)
    print clicks
    return prob_evaluate(interleaved, clicks)

    
N=4334
print "Probabilistic Interleaving"
print "Example:"
print " Rankings: ", L[N]
pair = preprocess(L[N])
# print pair
res = prob(pair,click_model = "RCM", parameters = [r])
print " Evaluation: ", res

Probabilistic Interleaving
Example:
 Rankings:  [[0, 0, 1, 2, 2], [2, 1, 1, 1, 2]]
[('16', 1), ('01', 0), ('25', 1), ('17', 1), ('29', 1), ('18', 1), ('00', 0), ('12', 0), ('24', 0), ('23', 0)]
[0, 1, 2, 6]
 Evaluation:  0


# Step 5
Implement User Clicks Simulation

In [10]:
table = []
with open('YandexRelPredChallenge.txt', 'r') as f:
    for line in f:
        line = line.split()
        table.append(line)
f.closed

sum = 0
S = []
for line in table:
    if(line[2]) == 'Q':
        S.append([])
        S[-1].append(line[5:])
        S[-1].append([])
    if(line[2]) == 'C':
        c = line[-1] 
        for i in range(-1,-len(S) - 1, -1): #attribute the click to the last query that had this document as a result
            if c in S[i][0]:
                S[i][1].append(c)
                break;

for i in range(5):
    print S[i]

[['7', '103', '51', '92', '43', '12', '73', '69', '27', '105'], []]
[['1625', '1627', '1623', '1626', '1624', '1622', '1619', '1621', '1620', '1618'], []]
[['2094', '2091', '2087', '2089', '2093', '2088', '2090', '2092', '2095', '2086'], []]
[['1625', '1627', '1623', '1626', '1624', '1622', '1619', '1621', '1620', '1618'], []]
[['17562', '1627', '1626', '1623', '2091', '17559', '17563', '17558', '17561', '17560'], ['17562', '1627', '1626']]


In [30]:
def RCM_parameter(S):
    sum1 = 0
    sum2 = 0
    for s in S:
        clicks = set()
        for c in s[1]:
            clicks.add(c)
        sum1 += len(clicks)

        sum2 += len(s[0])

    return 1.0*sum1/sum2

def RCM(l, r):
    c = []
    for i in range(len(l)):
        p = random.uniform(0, 1)
        if p < r:
            c.append(i)
    return c

r = RCM_parameter(S)
print "RCM parameter:", r

# for i in range(10):
#     print RCM([1,2,3,4,5,6,7,8,9,0], r)

RCM parameter: 0.126312951327


In [71]:
def get_Sr(queries, r):
    return  filter(lambda query: query[0][r-1] in query[1] , queries)

def get_lambda_r(queries, r):
    Sr = get_Sr(queries, r)
    n = len(Sr)
    sum = 0
    for s in Sr:
        indices = [-1] + [s[0].index(doc1) for doc1 in s[1]]
        last_clicked = indices[-1]+1
        additive = 0
        if last_clicked != r:
            additive = 1
        sum += additive
    return sum/float(len(Sr))
        
def get_lambdas(queries, maxr=10):
    return [get_lambda_r(queries, r) for r in range(1, maxr+1)]

def get_attractiveness(interleaved):
    rel_attr = {}
    rel_attr[0] = 0
    rel_attr[1] = 0.7
    rel_attr[2] = 0.9
    a = []
    for doc in interleaved:
        rel = int(doc[0][0])
        a.append(rel_attr[rel])
    return a

#get the clicks for the ranked document list, where
#'l_r' is the probability of continuing examination when the document at rank 'r' was clicked
#'a_u' is the attractiveness of document 'u' for this specific query
def SDCM(a_u,l_r):
    clicks = []
    for i in range(len(a_u)):
        p = random.uniform(0,1) #choose if to click
        if p < a_u[i]:
            clicks.append(i) 
            p = random.uniform(0,1) #choose if to stop examining
            if p < l_r[i]:
                continue
            else:
                break
    return clicks


lambdas = get_lambdas(S)
print "SDCM parameters (lambdas):"
for i in range(len(lambdas)):
    print (" l%d ="%i), lambdas[i]
# for i in range(10):
#     print SDCM([0.8, 0.9, 0.3, 0.5, 0.6, 0.4], lambdas)#[0.5, 0.6, 0.4, 0.3, 0.2, 0.3])

SDCM parameters (lambdas):
 l0 = 0.319429312581
 l1 = 0.53782013103
 l2 = 0.57064311896
 l3 = 0.583973655324
 l4 = 0.577966101695
 l5 = 0.557948717949
 l6 = 0.554520760574
 l7 = 0.505861919236
 l8 = 0.468378506895
 l9 = 0.227272727273


# Step 6
Simulate Interleaving Experiment 

In [31]:
interleavings = ['TeamDraft', 'Probabilistic']
clickModels = ['RCM', 'SDCM']


def run_simulation(pair, N):
    result = []
    for interleaving in interleavings:
        int_result = []
        
        #generate interleaved list
        if interleaving == 'TeamDraft':
            interleaved = team_draft_interleave(pair)
        elif interleaving =='Probabilistic':
            interleaved = prob_interleave(pair) 
            
        for clickModel in clickModels:
            sum = 0
            
            #prepare parameters of the selected click model
            parameters = []
            if clickModel == 'RCM':
                parameters = [r]
            elif clickModel == 'SDCM':
                parameters = lambdas
            
            #generate N samples
            for i in range(N):
                #generate clicks according to the selected click model
                clicks = simulate_clicks(interleaved,click_model = clickModel, parameters = parameters)

                #evaluate the score of the interleaved list with the generated clicks
                if interleaving == 'TeamDraft':
                    score = team_draft_evaluate(interleaved, clicks)
                    if score == -1:
                        sum += 1
                elif interleaving =='Probabilistic':
                    score = prob_evaluate(interleaved, clicks)
                    if score == -1:
                        sum += 1                    
            int_result.append(sum/float(N))
        result.append(int_result)
        
    return result

#run the simulation with all possible pairs of relevance
def run( N):
    result = []
    for pair in L:
        result.append(run_simulation(preprocess(pair), N))
    return result
    
# run_simulation(preprocess(L[5521]), 3)
simulation_results = run(3)

In [19]:
numpy.random.choice(5, 1, p=[0.1, 0, 0.3, 0.6, 0])[0]

3

0
